In [59]:
import os
import pickle
import numpy as np
import re
import emoji
import regex

In [60]:
def extract_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [61]:
def delete_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            
            text = re.sub("|".join(word), "", text)

    return text

In [62]:
# load emoji list
fullfile = os.path.expanduser("~/Dropbox/insight/Emoji/"+'mySmileys.p')
with open(fullfile, 'rb') as fp:
    emoji_list = pickle.load(fp)
print(emoji_list)
print(len(emoji_list))

['😀', '😁', '😂', '🤣', '😃', '😄', '😅', '😆', '😉', '😊', '😋', '😎', '😍', '😘', '😗', '😙', '😚', '☺️', '🙂', '🤗', '🤔', '😐', '😑', '😶', '🙄', '😏', '😣', '😥', '😮', '🤐', '😯', '😪', '😫', '😴', '😌', '😛', '😜', '😝', '🤤', '😒', '😓', '😔', '😕', '🙃', '🤑', '😲', '☹️', '🙁', '😖', '😞', '😟', '😤', '😢', '😭', '😦', '😧', '😨', '😩', '😬', '😰', '😱', '😳', '😵', '😡', '😠', '😷', '🤒', '🤕', '🤢', '🤧', '😇', '🤠', '🤡', '🤥', '🤓']
75


## sanitize tweets

In [5]:
def sanitize_tweets(tweet):
    # delete twitter-specific tags
    b1 = re.findall(r'@\S*', tweet) # Twitter user names
    b2 = re.findall(r'RT\S*', tweet) # "RT"
    b3 = re.findall(r'http\S*', tweet) # links

    c = tweet #list(tweet.split())
    for b in b1:
        c = re.sub(re.escape(b),"",c)
    for b in b2:
        c = re.sub(re.escape(b),"",c)
    for b in b3:
        c = re.sub(re.escape(b),"",c)
    return c
    
tweet = "RT @aa RT @bbb https:__ and other words! httpsaa" #RawT[-1]
sanitize_tweets(tweet)

'     and other words! '

In [64]:
# preprocessing with removing repeat tweets
# IN
target_names = emoji_list # ['😍','😡'] # 

# OUT
rawdata = []
data = []
raw_target = []
Len = []
filenames = []
target_arr = np.zeros((5000*75*2, 75)) # init bigger than actual
search_emoji = []

# main loop
ii = 0
for keyword in target_names:
    print(keyword)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
    with open(fullfile, 'rb') as fp:
        Tweets1 = pickle.load(fp)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
    with open(fullfile, 'rb') as fp:
        Tweets2 = pickle.load(fp)
    Tweets = Tweets1+Tweets2

    # remove repeat tweets
    myset = set(Tweets)
    Tweets_unique = list(myset)

    Text = []
    RawText = []
    E = []
    for tweet in Tweets_unique:
        # remove tweets with more than one smiley
#             emojis = extract_emojis(tweet)
#             a = set(emojis) & set(emoji_list)
#             a2 = list(a)
#             if len(a2)==1 and a2[0]==keyword: # else: deal with later if doing multi-D target

        # save raw tweet
        RawText.append(tweet)
        # save all emojis from input tweets
        emojis = extract_emojis(tweet)
        E.append(emojis)

        # save emojis into target array 
        for x in emojis:
            if x in emoji_list:
                ix = Edict[x]
                target_arr[ii,ix] = 1

        # save sanitized text
        text = sanitize_tweets(tweet)
        text = delete_emojis(text)
        Text.append(text)
        ii = ii+1

    data = data + Text
    rawdata = rawdata + RawText
    raw_target = raw_target + E
    filenames = filenames + [fullfile]
    search_emoji = search_emoji + [keyword]
    Len.append(len(Text))

😀
😁
😂
🤣
😃
😄
😅
😆
😉
😊
😋
😎
😍
😘
😗
😙
😚
☺️
🙂
🤗
🤔
😐
😑
😶
🙄
😏
😣
😥
😮
🤐
😯
😪
😫
😴
😌
😛
😜
😝
🤤
😒
😓
😔
😕
🙃
🤑
😲
☹️
🙁
😖
😞
😟
😤
😢
😭
😦
😧
😨
😩
😬
😰
😱
😳
😵
😡
😠
😷
🤒
🤕
🤢
🤧
😇
🤠
🤡
🤥
🤓


## 2. Make 75-dim target vector

In [49]:
N = []
for ii in range(75):
    N.append(ii)
Edict = dict(zip(emoji_list, N))
print(Edict)

{'😀': 0, '😁': 1, '😂': 2, '🤣': 3, '😃': 4, '😄': 5, '😅': 6, '😆': 7, '😉': 8, '😊': 9, '😋': 10, '😎': 11, '😍': 12, '😘': 13, '😗': 14, '😙': 15, '😚': 16, '☺️': 17, '🙂': 18, '🤗': 19, '🤔': 20, '😐': 21, '😑': 22, '😶': 23, '🙄': 24, '😏': 25, '😣': 26, '😥': 27, '😮': 28, '🤐': 29, '😯': 30, '😪': 31, '😫': 32, '😴': 33, '😌': 34, '😛': 35, '😜': 36, '😝': 37, '🤤': 38, '😒': 39, '😓': 40, '😔': 41, '😕': 42, '🙃': 43, '🤑': 44, '😲': 45, '☹️': 46, '🙁': 47, '😖': 48, '😞': 49, '😟': 50, '😤': 51, '😢': 52, '😭': 53, '😦': 54, '😧': 55, '😨': 56, '😩': 57, '😬': 58, '😰': 59, '😱': 60, '😳': 61, '😵': 62, '😡': 63, '😠': 64, '😷': 65, '🤒': 66, '🤕': 67, '🤢': 68, '🤧': 69, '😇': 70, '🤠': 71, '🤡': 72, '🤥': 73, '🤓': 74}


In [54]:
# 75-D preprocessing
# IN
target_names = emoji_list 

# OUT
rawdata = []
data = []
raw_target = []
Len = []
filenames = []
target_arr = np.zeros((5000*75*2, 75))
# search_emoji = []

# main loop
ii = 0
for keyword in target_names:
    print(keyword)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
    with open(fullfile, 'rb') as fp:
        Tweets1 = pickle.load(fp)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k_batch2/"+'outfile'+keyword)
    with open(fullfile, 'rb') as fp:
        Tweets2 = pickle.load(fp)
    Tweets = Tweets1 + Tweets2
    
    Len.append(len(Tweets))

    Text = []
    RawText = []
    E = []
    for tweet in Tweets:
        # save raw tweet
        RawText.append(tweet)
        # take out all emojis from input tweets
        emojis = extract_emojis(tweet)
        E.append(emojis)

        # save emojis into target array 
        for x in emojis:
            if x in emoji_list:
                ix = Edict[x]
                target_arr[ii,ix] = 1

        # save sanitized text
        text = sanitize_tweets(tweet)
        text = delete_emojis(text)
        Text.append(text)
        ii = ii+1

    data = data + Text
    rawdata = rawdata + RawText
    raw_target = raw_target + E
    filenames = filenames + [fullfile]
    # search_emoji = search_emoji + [keyword]

😀
😁
😂
🤣
😃
😄
😅
😆
😉
😊
😋
😎
😍
😘
😗
😙
😚
☺️
🙂
🤗
🤔
😐
😑
😶
🙄
😏
😣
😥
😮
🤐
😯
😪
😫
😴
😌
😛
😜
😝
🤤
😒
😓
😔
😕
🙃
🤑
😲
☹️
🙁
😖
😞
😟
😤
😢
😭
😦
😧
😨
😩
😬
😰
😱
😳
😵
😡
😠
😷
🤒
🤕
🤢
🤧
😇
🤠
🤡
🤥
🤓


In [56]:
# optional now: get 1-D target
numEmojis = len(target_names)
arr = []
for i in range(numEmojis):
    arr.extend([i] * Len[i]) 
target_1d = np.array(arr, dtype=int)

In [68]:
print(Len)

[2381, 3461, 3589, 2761, 3273, 3360, 2756, 2417, 3450, 3475, 3531, 3196, 3345, 3989, 3030, 3424, 3041, 3157, 3260, 3579, 2344, 3424, 3865, 1790, 4105, 3445, 3459, 3532, 2805, 1628, 2968, 3990, 3312, 3365, 3520, 3661, 3137, 3479, 3994, 3938, 3461, 3963, 3372, 3764, 2958, 2541, 3811, 3300, 3851, 3985, 2893, 3102, 2564, 2507, 2481, 3254, 3609, 3153, 3476, 1768, 1522, 2492, 3095, 2437, 3610, 3384, 3747, 1025, 3844, 3108, 3056, 2783, 2238, 3359, 3474]


In [69]:
# this struct is saving relatively raw data, for later use ~ visualizations
class tweet_data:
    pass

D = tweet_data()
D.raw_data = rawdata
D.data = data
D.raw_target = raw_target
D.filenames = filenames
D.numTweets = Len
D.target_1d = target_1d
D.target_arr = target_arr
D.target_names = target_names

In [70]:
# 10/02
fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_data_75xunique_target_arr.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(D, fp)

In [42]:
fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_data_75xsingles.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(D, fp)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target_arr, test_size= 0.2, random_state=1)

NameError: name 'data' is not defined

In [44]:
# this will be directly loaded to train model
class train_data:
    pass

T = train_data()
T.data = data
# X = vectorizer.fit_transform(data)
# T.X = X
# T.feature_names = vectorizer.get_feature_names()
# T.X_shape = X.shape
# T.Y = target_arr
T.y = target_1d
T.target_names = target_names

In [45]:


fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'train_data_75xsingles.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(T, fp)
